In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from trajkit import Trajectory, TrajectorySet

csv_path = "/Users/mehdi/particleTraj/res_xyti_time11.csv"  # update to your path

df_whole = pd.read_csv(csv_path)
df = df_whole[df_whole['t'] < 10]  # use a subset for faster loading
ts = TrajectorySet.from_dataframe(
    df,
    dataset_id="your_dataset",
    track_id_col="id",               # track identifier column
    position_cols=["x", "y"],        # coordinate columns
    time_col="t",                    # or use frame_col + frame_rate_hz
    frame_col="t",                  # set to your frame column if you have one
    units={"t": "frame", "x": "pixel"},
)
ts.summary_table().head()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from trajkit import TrajectorySet
from trajkit.cdv import run_cdv

csv_path = "/Users/mehdi/particleTraj/res_xyti_time11.csv"  # update to your path

df_whole = pd.read_csv(csv_path)
df = df_whole[df_whole['t'] < 10]  # use a subset for faster loading
ts = TrajectorySet.from_dataframe(
    df,
    dataset_id="your_dataset",
    track_id_col="id",               # track identifier column
    position_cols=["x", "y"],        # coordinate columns
    time_col="t",                    # or use frame_col + frame_rate_hz
    frame_col="t",                  # set to your frame column if you have one
    units={"t": "frame", "x": "pixel"},
)
ts.summary_table().head()
# Build TrajectorySet (t treated as frame index; frame_rate_hz=1.0 => time_seconds == frame)
ts = TrajectorySet.from_dataframe(
    df,
    dataset_id="demo",
    track_id_col="id",
    position_cols=["x", "y"],
    frame_col="t",
    frame_rate_hz=1.0,
    units={"t": "frame", "x": "pixel"},
)

# Grid in relative-position space
x = np.linspace(-300.0, 300.0, 50)
y = np.linspace(-300.0, 300.0, 50)
X, Y = np.meshgrid(x, y, indexing="xy")
grid_centers = np.stack([X.ravel(), Y.ravel()], axis=1)

# Run CDV end-to-end: build displacements (delta_t frames), pair/rotate, accumulate
mean, weights, counts, centers = run_cdv(
    ts,
    delta_t=1.0,           # delta_t in frames (since frame_rate_hz=1.0)
    grid_centers=grid_centers,
    time_tol=1e-3,
    kernel=50.0,           # hard cutoff radius in rel_pos space
    max_pair_distance=500, # optional pair distance filter; set None to disable
)

mean.shape, weights.sum(), counts.sum() if counts is not None else 0


In [ ]:
centers.shape

In [ ]:
import matplotlib.pyplot as plt
U = mean[:, 0].reshape(X.shape)
V = mean[:, 1].reshape(Y.shape)
mag = np.sqrt(U**2 + V**2) + 1e-9  # avoid zero division
U /= mag
V /= mag
plt.figure(figsize=(8, 8))
cs = plt.contourf(X, Y, np.log10(mag), levels=20, cmap="viridis")
plt.colorbar(cs, label="Displacement magnitude (pixels)")
plt.streamplot(X, Y, U, V, color="white", density=2)
